![imagenes](logo.png)

## Aplicación

Descargue desde https://github.com/scidatmath2020/Miscelanea/tree/main/R/Estadistica%20y%20Ciencia%20de%20datos/percepcion_corrupcion_autoridades_2022 los archivos ``Mexico_division_politica.csv``, ``tper_vic.RData`` y ``descriptor_autoridades.csv``.

## Problema

En la **ENVIPE 2022** del INEGI, se le pregunta al encuestado, entre otras cosas, si considera que ciertas autoridades son corruptas.

Dentro de la tabla ``tper_vic.RData``, en las columnas ``AP5_3_X``, con ``X`` tomando los valores 01, 02, 03,..., 10, se le pregunta al encuestado si reconoce a la autoridad X, donde las autoridades se indican más abajo.

En caso de respuesta afirmativa (valor 1), se le procede a preguntar si considera que la autoridad ``X`` es o no corrupta, siendo 1 la respuesta "Sí". Estas respuestas se encuentran en ``AP5_5_X``

El problema consiste en, para cada autoridad, generar una gráfica del mapa político de México coloreando cada entidad mediante el porcentaje de personas que reconocen dicha autoridad y la consideran corrupta respecto del total del personas que la reconocen.

In [ ]:
setwd("C:\\Users\\hp master\\OneDrive\\Escritorio\\mapa_corrupcion")

In [ ]:
load("tper_vic.RData")

In [ ]:
names(tper_vic)

In [ ]:
tapply(tper_vic$FAC_ELE,tper_vic$CVE_ENT,sum)

In [ ]:
library(tidyverse)

## Reconocimiento de autoridades

In [ ]:
autoridades = tper_vic %>% select(CVE_ENT,
                                  starts_with("AP5_3"),
                                  FAC_ELE)

In [ ]:
autoridades_tidy = autoridades %>% pivot_longer(names_to = "Autoridad",
                             values_to = "valor_reconocimiento",
                             cols = starts_with("AP5_3")
                            ) %>%
    filter(valor_reconocimiento == 1) %>%
    group_by(CVE_ENT,Autoridad) %>% 
    summarize(total_reconocimiento = sum(FAC_ELE))

## Concepción de corrupción

In [ ]:
corrupcion = tper_vic %>% select(CVE_ENT,
                                  starts_with("AP5_5"),
                                  FAC_ELE)

In [ ]:
corrupcion_tidy = corrupcion %>% pivot_longer(names_to = "corrupcion_autoridad",
                             values_to = "valor_corrupcion",
                             cols = starts_with("AP5_5")
                            ) %>%
    filter(valor_corrupcion == 1) %>%
    group_by(CVE_ENT,corrupcion_autoridad) %>% 
    summarize(total_corrupcion = sum(FAC_ELE))

## Cálculos de porcentajes

In [ ]:
autoridades_tidy

In [ ]:
corrupcion_tidy

In [ ]:
descriptor = read.csv("descriptor_autoridades.csv")

In [ ]:
descriptor

In [ ]:
informacion = cbind(autoridades_tidy,corrupcion_tidy) %>%
                mutate(porcentaje = total_corrupcion*100/total_reconocimiento) %>%
                left_join(descriptor, by = c("Autoridad"="CVE_Autoridad"))  %>%
                select(Entidad=CVE_ENT...1,Autoridad = Autoridad.y,porcentaje)

In [ ]:
informacion

## Mapa

In [ ]:
mapa = read.csv("Mexico_division_politica.csv")

In [ ]:
#mapa %>% filter(as.integer(Grupo) == 2)

mexico = mapa %>% mutate(Entidad = as.integer(Grupo))

In [ ]:
mapa_final = mexico %>% left_join(informacion)

In [ ]:
mapa_final %>% names()

In [ ]:
ggplot() +
    geom_polygon(data = mapa_final,
                 mapping = aes(x=Longitud,
                               y=Latitud,
                               group=Grupo,
                               fill = porcentaje)) +
   scale_fill_gradient(low = "yellow",
                      high = "darkred",
                      na.value = NA,
                      name="Porcentaje de mayores de edad\nque perciben corrupción en la autoridad") +
  coord_map(xlim=c(-117,-87),
            ylim=c(14,32)) +
  theme(legend.position="top",
        panel.background = element_rect("#202020"), 
        panel.grid = element_blank(),
        axis.title = element_blank(), 
        axis.text = element_blank(), 
        axis.ticks = element_blank(),
        plot.title = element_text(face="bold.italic")
  ) +
  facet_wrap(.~Autoridad,ncol=5) +
  labs(title = "Población de 18 años y más que identifica a las autoridades de seguridad\npública por entidad federativa y tipo de autoridad,\nsegún percepción de corrupción\nmarzo y abril de 2022", 
       subtitle = "Fuente: ENVIPE 2022, INEGI")
    
    